> Please run this in python310 and with a spacy kernel.

## Basic Usage

In [106]:
import time
from termcolor import colored
import coreferee, spacy
from spacy import displacy

In [174]:
trf_nlp = spacy.load('en_core_web_trf')
sm_nlp = spacy.load('en_core_web_sm')
lg_nlp = spacy.load('en_core_web_lg')
trf_nlp.add_pipe('coreferee')
sm_nlp.add_pipe('coreferee')
lg_nlp.add_pipe('coreferee')

In [154]:
doc = sm_nlp("Harry wanted to cook a meal for Sally. But she had to go home. He decided to go for a walk.")
doc._.coref_chains.print()

0: Harry(0), He(16)
1: Sally(7), she(10)


In [155]:
doc_trf = trf_nlp("Harry wanted to cook a meal for Sally. But she had to go home. He decided to go for a walk.")
doc_trf._.coref_chains.print()

0: Harry(0), He(16)
1: Sally(7), she(10)


## Utility Functions

In [156]:
COLORS = ['green', 'red', 'yellow', 'blue', 'magenta', 'cyan']

def get_color(index):
    return COLORS[index % len(COLORS)]

def cprint(text, color, newline=True):
    print(colored(text, color), end='\n' if newline else ' ')
    
def get_repr_from_mention(mention, prediction):
    if len(mention) > 1:
        return f'{prediction[mention[0]:mention[1] + 1]} - [{mention[0]}, {mention[1]}]'
    else:
        return f'{prediction[mention[0]]} - [{mention[0]}, {mention[0]}]'
    
def render_clusters(prediction):
    corefs = prediction._.coref_chains
    for i, cluster in enumerate(corefs):
        antecedent = get_repr_from_mention(cluster[cluster.most_specific_mention_index], prediction)
        cprint(f'Cluster {i} - Antecedent: {antecedent}', get_color(i))
        for mention in cluster:
            phrase = get_repr_from_mention(mention, prediction)
            cprint(phrase, get_color(i))
            
def predict_and_time(document, pipeline):
    start = time.time()
    prediction = pipeline(document)
    end = time.time()
    print(document)
    render_clusters(prediction)
    print(f"Elapsed Time: {(end - start) * 1000}ms");

In [157]:
predict_and_time("Although he was very busy with his work, Peter had had enough of it. He and his wife decided they needed a holiday. They travelled to Spain because they loved the country very much.", trf_nlp)

Although he was very busy with his work, Peter had had enough of it. He and his wife decided they needed a holiday. They travelled to Spain because they loved the country very much.
Cluster 0 - Antecedent: Peter - [9, 9]
he - [1, 1]
his - [6, 6]
Peter - [9, 9]
He - [16, 16]
his - [18, 18]
Cluster 1 - Antecedent: work - [7, 7]
work - [7, 7]
it - [14, 14]
Cluster 2 - Antecedent: He and his wife - [16, 19]
He and his wife - [16, 19]
they - [21, 21]
They - [26, 26]
they - [31, 31]
Cluster 3 - Antecedent: Spain - [29, 29]
Spain - [29, 29]
country - [34, 34]
Elapsed Time: 93.6577320098877ms


> Already much quicker than AllenNLP

## Excerpt Tests (`sm_nlp`)

In [158]:
short_excerpt = 'In practical terms, anarchy can refer to the curtailment or abolition of traditional forms of government and institutions. ' \
'It can also designate a nation or any inhabited place that has no system of government or central rule. ' \
'Anarchy is primarily advocated by individual anarchists who propose replacing government with voluntary institutions. ' \
'These institutions or free associations are generally modeled on nature.'

In [159]:
predict_and_time(short_excerpt, sm_nlp)

In practical terms, anarchy can refer to the curtailment or abolition of traditional forms of government and institutions. It can also designate a nation or any inhabited place that has no system of government or central rule. Anarchy is primarily advocated by individual anarchists who propose replacing government with voluntary institutions. These institutions or free associations are generally modeled on nature.
Cluster 0 - Antecedent: anarchy - [4, 4]
anarchy - [4, 4]
It - [20, 20]
Elapsed Time: 31.24690055847168ms


In [160]:
longer_article = 'Anarchy is a society without a government. It may also refer to a society or group of people that entirely rejects a set hierarchy. '\
'Anarchy was first used in English in 1539, meaning "an absence of government". Pierre-Joseph Proudhon adopted anarchy and anarchist in his 1840 treatise What Is Property? '\
'to refer to anarchism, a new political philosophy and social movement that advocates stateless societies based on free and voluntary associations. '\
'Anarchists seek a system based on the abolition of all coercive hierarchy, in particular the state, and many advocate for the creation of a system of direct democracy, '\
'worker cooperatives or privatization. In practical terms, anarchy can refer to the curtailment or abolition of traditional forms of government and institutions. '\
'It can also designate a nation or any inhabited place that has no system of government or central rule. Anarchy is primarily advocated by individual anarchists '\
'who propose replacing government with voluntary institutions. These institutions or free associations are generally modeled on nature since they can represent '\
'concepts such as community and economic self-reliance, interdependence, or individualism. Although anarchy is often negatively used as a synonym of chaos or '\
'societal collapse or anomie, this is not the meaning that anarchists attribute to anarchy, a society without hierarchies. '\
'Proudhon wrote that anarchy is "Not the Daughter But the Mother of Order."'

In [161]:
predict_and_time(longer_article, sm_nlp)

Anarchy is a society without a government. It may also refer to a society or group of people that entirely rejects a set hierarchy. Anarchy was first used in English in 1539, meaning "an absence of government". Pierre-Joseph Proudhon adopted anarchy and anarchist in his 1840 treatise What Is Property? to refer to anarchism, a new political philosophy and social movement that advocates stateless societies based on free and voluntary associations. Anarchists seek a system based on the abolition of all coercive hierarchy, in particular the state, and many advocate for the creation of a system of direct democracy, worker cooperatives or privatization. In practical terms, anarchy can refer to the curtailment or abolition of traditional forms of government and institutions. It can also designate a nation or any inhabited place that has no system of government or central rule. Anarchy is primarily advocated by individual anarchists who propose replacing government with voluntary institutions.

In [162]:
very_long_article = 'Following the outbreak of the civil war in Somalia and the ensuing collapse of the central government, residents reverted to '\
'local forms of conflict resolution, either secular, traditional or Islamic law, with a provision for appeal of all sentences. '\
'The legal structure in the country was divided along three lines, namely civil law, religious law and customary law (xeer). '\
'While Somalia\'s formal judicial system was largely destroyed after the fall of the Siad Barre regime, it was later gradually rebuilt and '\
'administered under different regional governments such as the autonomous Puntland and Somaliland macro-regions. '\
'In the case of the Transitional National Government and its successor the Transitional Federal Government, '\
'new interim judicial structures were formed through various international conferences. Despite some significant political differences between '\
'them, all of these administrations shared similar legal structures, much of which were predicated on the judicial systems of previous '\
'Somali administrations. These similarities in civil law included a charter which affirms the primacy of Muslim shari\'a or religious law, '\
'although in practice shari\'a is applied mainly to matters such as marriage, divorce, inheritance and civil issues. '\
'The charter assured the independence of the judiciary which in turn was protected by a judicial committee; a three-tier judicial system '\
'including a supreme court, a court of appeals and courts of first instance (either divided between district and regional courts, or a single court per region); '\
'and the laws of the civilian government which were in effect prior to the military coup d\'état that saw the Barre regime into power remain '\
'in forced until the laws are amended. '\
'Economist Alex Tabarrok claimed that Somalia in its stateless period provided a "unique test of the theory of anarchy", in some aspects near of '\
'that espoused by anarcho-capitalists such as David D. Friedman and Murray Rothbard. Nonetheless, both anarchists and some anarcho-capitalists '\
'such as Walter Block argue that Somalia was not an anarchist society.'

In [163]:
predict_and_time(very_long_article, sm_nlp)

Following the outbreak of the civil war in Somalia and the ensuing collapse of the central government, residents reverted to local forms of conflict resolution, either secular, traditional or Islamic law, with a provision for appeal of all sentences. The legal structure in the country was divided along three lines, namely civil law, religious law and customary law (xeer). While Somalia's formal judicial system was largely destroyed after the fall of the Siad Barre regime, it was later gradually rebuilt and administered under different regional governments such as the autonomous Puntland and Somaliland macro-regions. In the case of the Transitional National Government and its successor the Transitional Federal Government, new interim judicial structures were formed through various international conferences. Despite some significant political differences between them, all of these administrations shared similar legal structures, much of which were predicated on the judicial systems of pr

> Overall much faster than Allennlp, but there are some inconsistencies.

## Excerpt Tests (`trf_nlp`)

In [164]:
predict_and_time(short_excerpt, trf_nlp)

In practical terms, anarchy can refer to the curtailment or abolition of traditional forms of government and institutions. It can also designate a nation or any inhabited place that has no system of government or central rule. Anarchy is primarily advocated by individual anarchists who propose replacing government with voluntary institutions. These institutions or free associations are generally modeled on nature.
Cluster 0 - Antecedent: anarchy - [4, 4]
anarchy - [4, 4]
It - [20, 20]
Elapsed Time: 136.85345649719238ms


In [165]:
predict_and_time(longer_article, trf_nlp)

Anarchy is a society without a government. It may also refer to a society or group of people that entirely rejects a set hierarchy. Anarchy was first used in English in 1539, meaning "an absence of government". Pierre-Joseph Proudhon adopted anarchy and anarchist in his 1840 treatise What Is Property? to refer to anarchism, a new political philosophy and social movement that advocates stateless societies based on free and voluntary associations. Anarchists seek a system based on the abolition of all coercive hierarchy, in particular the state, and many advocate for the creation of a system of direct democracy, worker cooperatives or privatization. In practical terms, anarchy can refer to the curtailment or abolition of traditional forms of government and institutions. It can also designate a nation or any inhabited place that has no system of government or central rule. Anarchy is primarily advocated by individual anarchists who propose replacing government with voluntary institutions.

In [166]:
predict_and_time(very_long_article, trf_nlp)

Following the outbreak of the civil war in Somalia and the ensuing collapse of the central government, residents reverted to local forms of conflict resolution, either secular, traditional or Islamic law, with a provision for appeal of all sentences. The legal structure in the country was divided along three lines, namely civil law, religious law and customary law (xeer). While Somalia's formal judicial system was largely destroyed after the fall of the Siad Barre regime, it was later gradually rebuilt and administered under different regional governments such as the autonomous Puntland and Somaliland macro-regions. In the case of the Transitional National Government and its successor the Transitional Federal Government, new interim judicial structures were formed through various international conferences. Despite some significant political differences between them, all of these administrations shared similar legal structures, much of which were predicated on the judicial systems of pr

> There are still some inaccuracies here, maybe even more than `sm_nlp`. However, coreferee only references the head. Get the whole subtree.

Taking an example from the long article above, using `charter [174, 174]`.

In [167]:
prediction = sm_nlp(very_long_article)

In [168]:
' '.join(map(str, prediction[174].subtree))

"a charter which affirms the primacy of Muslim shari'a or religious law"

Honestly seems like the small pipeline is more accurate.

## Utility Functions Revisited

In [171]:
COLORS = ['green', 'red', 'yellow', 'blue', 'magenta', 'cyan']

def get_color(index):
    return COLORS[index % len(COLORS)]

def cprint(text, color, newline=True):
    print(colored(text, color), end='\n' if newline else ' ')
    
def get_repr_from_mention(mention, prediction):
    if len(mention) > 1:
        return f'{prediction[mention[0]:mention[1] + 1]} - mention indices [{mention[0]}, {mention[1]}]'
    else:
        left = prediction[mention[0]].left_edge.i
        right = prediction[mention[0]].right_edge.i
        merged_phrase_subtree = str(prediction[left:right + 1])
        return f'{merged_phrase_subtree} - mention indices [{mention[0]}, {mention[0]}] - subtree indices [{left}, {right}]'
    
def render_clusters(prediction):
    corefs = prediction._.coref_chains
    for i, cluster in enumerate(corefs):
        antecedent = get_repr_from_mention(cluster[cluster.most_specific_mention_index], prediction)
        cprint(f'Cluster {i} - {antecedent}', get_color(i))
        for mention in cluster:
            phrase = get_repr_from_mention(mention, prediction)
            cprint(phrase, get_color(i))
            
def predict_and_time(document, pipeline):
    start = time.time()
    prediction = pipeline(document)
    end = time.time()
    print(document)
    render_clusters(prediction)
    print(f"Elapsed Time: {(end - start) * 1000}ms");

## More Testing with Noun Phrases and `lg_nlp`

In [178]:
predict_and_time(very_long_article, lg_nlp)

Following the outbreak of the civil war in Somalia and the ensuing collapse of the central government, residents reverted to local forms of conflict resolution, either secular, traditional or Islamic law, with a provision for appeal of all sentences. The legal structure in the country was divided along three lines, namely civil law, religious law and customary law (xeer). While Somalia's formal judicial system was largely destroyed after the fall of the Siad Barre regime, it was later gradually rebuilt and administered under different regional governments such as the autonomous Puntland and Somaliland macro-regions. In the case of the Transitional National Government and its successor the Transitional Federal Government, new interim judicial structures were formed through various international conferences. Despite some significant political differences between them, all of these administrations shared similar legal structures, much of which were predicated on the judicial systems of pr

In [177]:
predict_and_time(very_long_article, sm_nlp)

Following the outbreak of the civil war in Somalia and the ensuing collapse of the central government, residents reverted to local forms of conflict resolution, either secular, traditional or Islamic law, with a provision for appeal of all sentences. The legal structure in the country was divided along three lines, namely civil law, religious law and customary law (xeer). While Somalia's formal judicial system was largely destroyed after the fall of the Siad Barre regime, it was later gradually rebuilt and administered under different regional governments such as the autonomous Puntland and Somaliland macro-regions. In the case of the Transitional National Government and its successor the Transitional Federal Government, new interim judicial structures were formed through various international conferences. Despite some significant political differences between them, all of these administrations shared similar legal structures, much of which were predicated on the judicial systems of pr

The large and small pipelines are comparable in speed, but the large pipeline is more accurate with the dependency parser, while the small pipeline found the coreferences better.

## Continued Testing with `lg_nlp`

In [181]:
predict_and_time(short_excerpt, lg_nlp)

In practical terms, anarchy can refer to the curtailment or abolition of traditional forms of government and institutions. It can also designate a nation or any inhabited place that has no system of government or central rule. Anarchy is primarily advocated by individual anarchists who propose replacing government with voluntary institutions. These institutions or free associations are generally modeled on nature.
Cluster 0 - anarchy - mention indices [4, 4] - subtree indices [4, 4]
anarchy - mention indices [4, 4] - subtree indices [4, 4]
It - mention indices [20, 20] - subtree indices [20, 20]
Elapsed Time: 46.814680099487305ms


In [182]:
predict_and_time(longer_article, lg_nlp)

Anarchy is a society without a government. It may also refer to a society or group of people that entirely rejects a set hierarchy. Anarchy was first used in English in 1539, meaning "an absence of government". Pierre-Joseph Proudhon adopted anarchy and anarchist in his 1840 treatise What Is Property? to refer to anarchism, a new political philosophy and social movement that advocates stateless societies based on free and voluntary associations. Anarchists seek a system based on the abolition of all coercive hierarchy, in particular the state, and many advocate for the creation of a system of direct democracy, worker cooperatives or privatization. In practical terms, anarchy can refer to the curtailment or abolition of traditional forms of government and institutions. It can also designate a nation or any inhabited place that has no system of government or central rule. Anarchy is primarily advocated by individual anarchists who propose replacing government with voluntary institutions.